In [1]:
from netCDF4 import Dataset
import numpy as np
from datetime import datetime, timedelta

In [2]:
import sys, os, pathlib

In [3]:
from tqdm import tqdm

In [4]:
def DoesPathExistAndIsDirectory(pathStr):
    if os.path.exists(pathStr) and os.path.isdir(pathStr):
        return True
    else:
        return False


def DoesPathExistAndIsFile(pathStr):
    if os.path.exists(pathStr) and os.path.isfile(pathStr):
        return True
    else:
        return False


def EnsureDirectoryExists(pathStr):
    if not DoesPathExistAndIsDirectory(pathStr):
        try:
            # os.mkdir(pathStr)
            pathlib.Path(pathStr).mkdir(parents=True, exist_ok=True)
            return True
        except Exception as ex:
            err_fname = './errors.log'
            exc_type, exc_value, exc_traceback = sys.exc_info()
            with open(err_fname, 'a') as errf:
                traceback.print_tb(exc_traceback, limit=None, file=errf)
                traceback.print_exception(exc_type, exc_value, exc_traceback, limit=None, file=errf)
            print(str(ex))
            print('the directory you are trying to place a file to doesn\'t exist and cannot be created:\n%s' % pathStr)
            raise FileNotFoundError('the directory you are trying to place a file to doesn\'t exist and cannot be created:')
    else:
        return True

In [5]:
ds = Dataset('./ERA5_slp.nc', 'r')

In [6]:
ds.variables

OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
              float32 longitude(longitude)
                  units: degrees_east
                  long_name: longitude
              unlimited dimensions: 
              current shape = (1440,)
              filling off), ('latitude', <class 'netCDF4._netCDF4.Variable'>
              float32 latitude(latitude)
                  units: degrees_north
                  long_name: latitude
              unlimited dimensions: 
              current shape = (721,)
              filling off), ('time', <class 'netCDF4._netCDF4.Variable'>
              int32 time(time)
                  units: hours since 1900-01-01 00:00:00.0
                  long_name: time
                  calendar: gregorian
              unlimited dimensions: 
              current shape = (24,)
              filling off), ('msl', <class 'netCDF4._netCDF4.Variable'>
              int16 msl(time, latitude, longitude)
                  scale_factor: 0.14212686

In [7]:
datetimes_int = ds.variables['time'][:]
datetimes_int = [int(t) for t in datetimes_int]

In [8]:
dt0 = datetime(1900, 1, 1)

In [9]:
datetimes = [dt0 + timedelta(hours=d) for d in datetimes_int]

In [10]:
datetimes[0]

datetime.datetime(2004, 1, 30, 0, 0)

In [11]:
slp_data = ds.variables['msl'][:]
slp_data.shape

(24, 721, 1440)

In [12]:
lats = ds.variables['latitude'][:]
lats.shape

(721,)

In [13]:
lons = ds.variables['longitude'][:]
lons.shape

(1440,)

In [14]:
ds.close()

In [15]:
del ds

In [16]:
dt_idx = 0
dt = datetimes[dt_idx]

In [17]:
dt

datetime.datetime(2004, 1, 30, 0, 0)

In [19]:
fname = './%04d_split/era5_SLP-%04d-%02d-%02d-%02d.nc' % (dt.year, dt.year, dt.month, dt.day, dt.hour)
fname

'./2004_split/era5_SLP-2004-01-30-00.nc'

In [21]:
for dt_idx in tqdm(range(len(datetimes_int)), total=len(datetimes_int)):
    dt = datetimes[dt_idx]
    directory = './slp/%04d_split/' % (dt.year)
    EnsureDirectoryExists(directory)
    fname = os.path.join(directory, 'era5_SLP-%04d-%02d-%02d-%02d.nc' % (dt.year, dt.month, dt.day, dt.hour))
    
    ds_out = Dataset(fname, 'w')
    datetimedim = ds_out.createDimension('time_dim', size = 1)
    lats_dim = ds_out.createDimension('lat_dim', size = lats.shape[0])
    lons_dim = ds_out.createDimension('lon_dim', size = lons.shape[0])
    datetimes_var = ds_out.createVariable('time','int32',('time_dim',))
    lats_var = ds_out.createVariable('latitude','float32',('lat_dim',))
    lons_var = ds_out.createVariable('longitude','float32',('lon_dim',))
    data_array = ds_out.createVariable('msl','float32',('time_dim','lat_dim','lon_dim',),least_significant_digit=17)
    ds_out.description = 'SLP sliced data'
    ds_out.history = 'Created ' + str(datetime.now())
    lats_var.units = 'degrees north'
    lons_var.units = 'degrees east'
    datetimes_var.units = 'hours since 1900-01-01 00:00:00.0'
    datetimes_var.calendar = 'gregorian'

    lats_var[:] = lats
    lons_var[:] = lons
    datetimes_var[:] = datetimes_int[dt_idx]
    data_array[:] = slp_data[dt_idx]

    ds_out.close()

100%|██████████| 24/24 [00:01<00:00, 22.43it/s]
